# VanMoof's Capital Bikeshare Project - EDA Data Cleaning Notebook

#### Developed by Clare Cruz

## Package Import

In [1]:
import pandas as pd
import datetime 
import numpy as np

pd.set_option("display.max_rows", 10)

## Data Import

We are bringing in the data from May 2019 and June 2019. These dates were chosen to avoid complications from COVID and a proof of concept for the modeling process. <br>
We are also bringing in the station capacity data from https://opendata.dc.gov/ which provides data as of the pull date, which was 10/25/2021

In [3]:
june_bike_data = pd.read_csv('Data/201906-capitalbikeshare-tripdata.csv')
bike_capacity = pd.read_csv('Data/Capital_Bike_Share_Locations.csv')
may_bike_data = pd.read_csv('Data/201905-capitalbikeshare-tripdata.csv')

#### Datatime formatting

To make life easier, we will format the date data to Python's datetime type

Please note that for now, we will treat the may data and the remaining months data separately so that we can separately calculate the starting number of bikes. 

In [4]:
# Transform the data to pandas date time variables
may_bike_data['Start date'] = pd.to_datetime(may_bike_data['Start date'], format = '%Y-%m-%d %H:%M:%S')
may_bike_data['End date'] = pd.to_datetime(may_bike_data['End date'], format = '%Y-%m-%d %H:%M:%S')

june_bike_data['Start date'] = pd.to_datetime(june_bike_data['Start date'], format = '%Y-%m-%d %H:%M:%S')
june_bike_data['End date'] = pd.to_datetime(june_bike_data['End date'], format = '%Y-%m-%d %H:%M:%S')

#### Extract the day info and time info separately

In [5]:
# Extract the start and end times and date into separate columns
may_bike_data['Start_time'] = may_bike_data['Start date'].dt.time
may_bike_data['End_time'] = may_bike_data['End date'].dt.time

may_bike_data['Start_day'] = may_bike_data['Start date'].dt.date
may_bike_data['End_day'] = may_bike_data['End date'].dt.date

june_bike_data['Start_time'] = june_bike_data['Start date'].dt.time
june_bike_data['End_time'] = june_bike_data['End date'].dt.time

june_bike_data['Start_day'] = june_bike_data['Start date'].dt.date
june_bike_data['End_day'] = june_bike_data['End date'].dt.date

In [6]:
# Define hour 
may_bike_data['start_hour'] = may_bike_data['Start date'].dt.hour
june_bike_data['start_hour'] = june_bike_data['Start date'].dt.hour
#raw_bike_data

### Accounting for the reshuffling

One of the issues in this project is the van reshuffling. Our solution for this is to add rows to the data to account for these transitions. The outline of the reshuffling looks like: <br>
For one bike: <br>
Trip 1 - Station A --> Station B <br>
Trip 2 - Station B --> Van (**Add row**) <br>
Trip 3 - Van --> Station C (**Add row**) <br>
Trip 4 - Station C --> Station D

In [445]:
# Drop the rows that have a blank bike number (there are a few)
sorted_bike_data = june_bike_data[june_bike_data['Bike number'].notna()]
shuffle_bike_data = sorted_bike_data.sort_values(['Bike number', 'Start date'])
shuffle_bike_data
# Reset the index so that our indices work in the for loop in the next chunk
shuffle_bike_data.reset_index(inplace = True, drop = True)

In [446]:
# Adds two rows each time the van has reshuffled the data
# A --> B
# B --> Van
# Van --> C
# C -->
count = 0
# For every row in the dataset
shuffle_bike_data_clean = []
for i in range(len(shuffle_bike_data)-1):
    if shuffle_bike_data.loc[i,'End station number'] != shuffle_bike_data.loc[i+1,'Start station number']:
        if shuffle_bike_data.loc[i,'Bike number'] == shuffle_bike_data.loc[i+1,'Bike number']:
            count += 1
            # Add the two rows to the dataframe
            shuffle_bike_data_clean.append([0, 
                                            shuffle_bike_data.loc[i,'End date'], 
                                            shuffle_bike_data.loc[i,'End date'], 
                                            shuffle_bike_data.loc[i,'End station number'],
                                            shuffle_bike_data.loc[i,'End station'], 
                                            'VAN001', 
                                            'VAN', 
                                            shuffle_bike_data.loc[i,'Bike number'],
                                            'NA',
                                            shuffle_bike_data.loc[i,'End_time'],
                                            shuffle_bike_data.loc[i,'End_time'],
                                            shuffle_bike_data.loc[i,'End_day'],
                                            shuffle_bike_data.loc[i,'End_day'], 
                                            shuffle_bike_data.loc[i,'start_hour']])
            shuffle_bike_data_clean.append([0,
                                            shuffle_bike_data.loc[i,'End date'],
                                            shuffle_bike_data.loc[i,'End date'],
                                            'VAN001',
                                            'VAN',
                                            shuffle_bike_data.loc[i+1,'Start station number'],
                                            shuffle_bike_data.loc[i+1,'Start station'],
                                            shuffle_bike_data.loc[i,'Bike number'],
                                            'NA',
                                            shuffle_bike_data.loc[i,'End_time'],
                                            shuffle_bike_data.loc[i,'End_time'], 
                                            shuffle_bike_data.loc[i,'End_day'],
                                            shuffle_bike_data.loc[i,'End_day'], 
                                            shuffle_bike_data.loc[i,'start_hour']])

In [447]:
# Make the new rows a dataframe
van_data = pd.DataFrame(shuffle_bike_data_clean, columns=['Duration', 'Start date', 'End date', 'Start station number', 'Start station', 'End station number', 'End station', 'Bike number', 'Member type','Start_time', 'End_time', 'Start_day', 'End_day', 'start_hour'])
van_data.shape

(65200, 14)

In [448]:
# Drop the rows that have a blank bike number (there are a few)
may_sorted_bike_data = may_bike_data[may_bike_data['Bike number'].notna()]
may_shuffle_bike_data = may_sorted_bike_data.sort_values(['Bike number', 'Start date'])
#shuffle_bike_data
# Reset the index so that our indices work in the for loop in the next chunk
may_shuffle_bike_data.reset_index(inplace = True, drop = True)

In [449]:
# Adds two rows each time the van has reshuffled the data
# A --> B
# B --> Van
# Van --> C
# C -->
count = 0
# For every row in the dataset
shuffle_bike_data_clean = []
for i in range(len(may_shuffle_bike_data)-1):
    if may_shuffle_bike_data.loc[i,'End station number'] != may_shuffle_bike_data.loc[i+1,'Start station number']:
        if may_shuffle_bike_data.loc[i,'Bike number'] == may_shuffle_bike_data.loc[i+1,'Bike number']:
            count += 1
            # Add the two rows to the dataframe
            shuffle_bike_data_clean.append([0, 
                                            may_shuffle_bike_data.loc[i,'End date'], 
                                            may_shuffle_bike_data.loc[i,'End date'], 
                                            may_shuffle_bike_data.loc[i,'End station number'],
                                            may_shuffle_bike_data.loc[i,'End station'], 
                                            'VAN001', 
                                            'VAN', 
                                            may_shuffle_bike_data.loc[i,'Bike number'],
                                            'NA',
                                            may_shuffle_bike_data.loc[i,'End_time'],
                                            may_shuffle_bike_data.loc[i,'End_time'],
                                            may_shuffle_bike_data.loc[i,'End_day'],
                                            may_shuffle_bike_data.loc[i,'End_day'], 
                                            may_shuffle_bike_data.loc[i,'start_hour']])
            shuffle_bike_data_clean.append([0,
                                            may_shuffle_bike_data.loc[i,'End date'],
                                            may_shuffle_bike_data.loc[i,'End date'],
                                            'VAN001',
                                            'VAN',
                                            may_shuffle_bike_data.loc[i+1,'Start station number'],
                                            may_shuffle_bike_data.loc[i+1,'Start station'],
                                            may_shuffle_bike_data.loc[i,'Bike number'],
                                            'NA',
                                            may_shuffle_bike_data.loc[i,'End_time'],
                                            may_shuffle_bike_data.loc[i,'End_time'], 
                                            may_shuffle_bike_data.loc[i,'End_day'],
                                            may_shuffle_bike_data.loc[i,'End_day'], 
                                            may_shuffle_bike_data.loc[i,'start_hour']])

In [450]:
# Make the new rows a dataframe
may_van_data = pd.DataFrame(shuffle_bike_data_clean, columns=['Duration', 'Start date', 'End date', 'Start station number', 'Start station', 'End station number', 'End station', 'Bike number', 'Member type','Start_time', 'End_time', 'Start_day', 'End_day', 'start_hour'])
may_van_data.shape

(64238, 14)

In [451]:
# Combine the van data
print(shuffle_bike_data.shape)
june_shuffle_data = pd.concat([shuffle_bike_data, van_data], axis = 0)
june_shuffle_data.shape

(350130, 14)


(415330, 14)

Data has been successfully added

In [452]:
# Combine the van data
print(may_shuffle_bike_data.shape)
may_shuffle_data = pd.concat([may_shuffle_bike_data, may_van_data], axis = 0)
may_shuffle_data.shape

(337704, 14)


(401942, 14)

Data has been successfully added

#### Partition data into two tables: departures and arrivals

Calculating the number of arrivals and departures separately makes life easier.

In [453]:
# Make a ride id column so that we can merge two dfs together later
may_bike_data['ride_id'] = np.arange(may_bike_data.shape[0])
june_shuffle_data['ride_id'] = np.arange(june_shuffle_data.shape[0])

In [454]:
# Make two separate tables so we can easily calculate the number of arrivals/departures
may_arrivals = may_bike_data
may_departures = may_bike_data

june_arrivals = june_shuffle_data
june_departures = june_shuffle_data

#### Arrivals Table

In [455]:
# Aggregate the data
may_arrivals_agg = may_arrivals.groupby(['Start_day', 'start_hour', 'End station number']).count()
june_arrivals_agg = june_arrivals.groupby(['Start_day', 'start_hour', 'End station number']).count()

# We only need one column to get the counts 
may_arrivals_df = may_arrivals_agg['Bike number']
june_arrivals_df = june_arrivals_agg['Bike number']

# Move the start day, start hour, and station number variables to columns instead of indexes
may_arrivals_df = may_arrivals_df.reset_index(level = [0,1,2])
june_arrivals_df = june_arrivals_df.reset_index(level = [0,1,2])

# Normalize the variable names so we can merge tables
may_arrivals_df.rename({'End station number':'station_id', 'Bike number':'num_arrivals'}, axis = 1, inplace = True)
june_arrivals_df.rename({'End station number':'station_id', 'Bike number':'num_arrivals'}, axis = 1, inplace = True)
june_arrivals_df[june_arrivals_df['station_id'] == 31000].head()

,Start_day,start_hour,station_id,num_arrivals
3368,2019-06-01,19,31000,1
5099,2019-06-02,9,31000,1
5834,2019-06-02,12,31000,1
6111,2019-06-02,13,31000,1
7197,2019-06-02,17,31000,1


#### Departures Table

In [456]:
# Aggregate the data
may_depart_agg = may_departures.groupby(['Start_day', 'start_hour','Start station number']).count()
june_depart_agg = june_departures.groupby(['Start_day', 'start_hour','Start station number']).count()

# We only need one column to get the counts 
may_depart_df = may_depart_agg['Bike number']
june_depart_df = june_depart_agg['Bike number']

# Move the start day, start hour, and station number variables to columns instead of indexes
may_depart_df = may_depart_df.reset_index(level = [0,1,2])
june_depart_df = june_depart_df.reset_index(level = [0,1,2])

# Normalize the variable names so we can merge tables
may_depart_df.rename({'Start station number':'station_id', 'Bike number':'num_depart'}, axis = 1, inplace = True)
june_depart_df.rename({'Start station number':'station_id', 'Bike number':'num_depart'}, axis = 1, inplace = True)

#### Merge departure and arrivals table together

In [457]:
may_final_bike_data = pd.merge(may_arrivals_df, may_depart_df, how = 'outer', on = ['Start_day', 'start_hour','station_id']).fillna(0)
june_final_bike_data = pd.merge(june_arrivals_df, june_depart_df, how = 'outer', on = ['Start_day', 'start_hour','station_id']).fillna(0)

may_final_bike_data.reset_index(inplace = True)

In [458]:
june_final_bike_data.head()

,Start_day,start_hour,station_id,num_arrivals,num_depart
0,2019-06-01,0,31004,1.0,0.0
1,2019-06-01,0,31014,1.0,0.0
2,2019-06-01,0,31018,1.0,0.0
3,2019-06-01,0,31020,1.0,0.0
4,2019-06-01,0,31026,1.0,0.0


### Calculating the starting number of bikes at every station

Another quirk to this dataset is the starting number of bikes. If we start out in June without looking at some relative history, we are assuming that all of this stations have no bikes to start with. Therefore, we will look at May data to get a sense of how many bikes the stations start out with. We considered a whole month because some bikes don't get used very often so we need to expand the timeframe to capture all the bikes.

The starting number of bikes is calculated by finding the last location for each bike. A high level outline of the code is: <br>
<br>
For every bike: <br>
>    For every station:
>>        When was the last time the bike arrived and departed there
>>        If the departure time is before the arrival time
>>>            Then that bike must be at that station and flag it

In [459]:
may_shuffle_data.reset_index(drop = True, inplace = True)

In [460]:
# Get the latest time every bike was at every station
start_depart = may_shuffle_data[['Bike number', 'Start date', 'Start station number']].loc[may_shuffle_data.groupby(['Start station number','Bike number'])['Start date'].idxmax()]
start_arr = may_shuffle_data[['Bike number', 'End date', 'End station number']].loc[may_shuffle_data.groupby(['End station number','Bike number'])['End date'].idxmax()]

start_arr.reset_index(inplace = True, drop = True)
start_depart.reset_index(inplace = True, drop = True)

start_arr.rename({'End date':'max_arrived', 'End station number':'station_id'},axis = 1, inplace = True)
start_depart.rename({'Start date':'max_depart', 'Start station number':'station_id'},axis = 1, inplace = True)

#### There are bikes that are present in May but not in June so we need to exclude them

In [461]:
diff_bike = len(list(set(may_shuffle_data['Bike number'].value_counts().index)-set(june_shuffle_data['Bike number'].value_counts().index)))
print('Number of bikes that are in May but not june: {}'.format(diff_bike))

Number of bikes that are in May but not june: 218


In [462]:
# Excluse any bikes that are not present in June
start_arr_filtered = start_arr.loc[start_arr['Bike number'].isin(june_bike_data['Bike number'].value_counts().index)]
start_depart_filtered = start_depart.loc[start_depart['Bike number'].isin(june_bike_data['Bike number'].value_counts().index)]

In [463]:
# Merge the arrivals and departure together and create a flag that indicates the last station every bike is at
may_last_bike_full = pd.merge(start_arr_filtered,start_depart_filtered,  how = 'outer', left_on = ['Bike number','station_id'], right_on = ['Bike number','station_id'])
may_last_bike_full['flag'] = np.where(((may_last_bike_full.max_arrived > may_last_bike_full.max_depart) | (may_last_bike_full.max_depart.isna())) , 1, 0)
print('Number of bikes in May: {}'.format(len(may_shuffle_data['Bike number'].value_counts())))
print('Number of bikes in May after excluding bikes that are not present in June: {}'.format(may_last_bike_full[may_last_bike_full['flag'] == 1]['flag'].sum()))

Number of bikes in May: 4720
Number of bikes in May after excluding bikes that are not present in June: 4502


In [464]:
# Create a dataframe that matches the June data so that the tables can be combined
may_last_bike = may_last_bike_full.groupby('station_id')['flag'].sum()
may_last_bike = pd.DataFrame(may_last_bike)
may_last_bike.reset_index(inplace = True)
may_last_bike['Start_day'] = datetime.date(day = 31, month = 5, year = 2019) # Can be anything
may_last_bike['start_hour'] = 23 # Can be anything
may_last_bike['num_depart'] = 0 # So we can get the starting number
may_last_bike.rename({'flag':'num_arrivals'}, axis = 1,inplace = True)
may_last_bike['diff'] = may_last_bike['num_arrivals']
may_last_bike.head()

,station_id,num_arrivals,Start_day,start_hour,num_depart,diff
0,31000,6,2019-05-31,23,0,6
1,31001,2,2019-05-31,23,0,2
2,31002,4,2019-05-31,23,0,4
3,31003,1,2019-05-31,23,0,1
4,31004,5,2019-05-31,23,0,5


#### Calculate the difference in arrivals and departures

In [465]:
june_final_bike_data['diff'] = june_final_bike_data['num_arrivals'] - june_final_bike_data['num_depart']

In [466]:
full_data = pd.concat([may_last_bike, june_final_bike_data], axis = 0)
full_data.head()

,station_id,num_arrivals,Start_day,start_hour,num_depart,diff
0,31000,6.0,2019-05-31,23,0.0,6.0
1,31001,2.0,2019-05-31,23,0.0,2.0
2,31002,4.0,2019-05-31,23,0.0,4.0
3,31003,1.0,2019-05-31,23,0.0,1.0
4,31004,5.0,2019-05-31,23,0.0,5.0


### Bikeshare capacity data

Now we need the capacity information from the other dataset so that we can set a limit to the cumulative difference for bike availability

In [467]:
filtered_capacity = bike_capacity[['NAME','NUM_DOCKS_AVAILABLE','NUM_BIKES_AVAILABLE' ,'LONGITUDE', "LATITUDE"]]
filtered_capacity.head()

,NAME,NUM_DOCKS_AVAILABLE,NUM_BIKES_AVAILABLE,LONGITUDE,LATITUDE
0,Duke St & John Carlyle St,5,10,-77.060866,38.804378
1,Langston Blvd & N Cleveland St,8,7,-77.091690,38.894941
2,Arlington Blvd & S George Mason Dr,12,3,-77.104503,38.869442
3,10th & G St NW,15,7,-77.026235,38.898243
4,Langston Blvd & N Scott St,4,11,-77.080851,38.897612


In [468]:
# Get a dataframe with all the bike stations and their names
unique_stations = june_bike_data.drop_duplicates(subset=['Start station number'])[['Start station number', 'Start station']]

In [483]:
# Add back the station names and the capacity to the dataset
full_data_stations = pd.merge(full_data, unique_stations, how = 'left', left_on = 'station_id', right_on = 'Start station number')
full_bike_data = pd.merge(full_data_stations, filtered_capacity, how = 'left', left_on = 'Start station', right_on = 'NAME')

In [484]:
# Remove all the stations that are not in our dataset
full_bike_data = full_bike_data[full_bike_data['NAME'].notna()]

There's only 30 stations that do not have a capacity and all have either been moved, removed, or renamed

In [485]:
# Calculate the total capacity
full_bike_data['total_capacity'] = full_bike_data['NUM_DOCKS_AVAILABLE'] + full_bike_data['NUM_BIKES_AVAILABLE']

In [486]:
# Drop the redundant info 
full_bike_data.drop(columns = ['NAME', 'Start station number','NUM_DOCKS_AVAILABLE','NUM_BIKES_AVAILABLE'], inplace = True)
full_bike_data.rename({'Start station': 'station_name',  
                       'LONGITUDE': 'longitude', 
                       'LATITUDE':'latitude'} , axis = 1, inplace = True)

In [487]:
full_bike_data.head()

,station_id,num_arrivals,Start_day,start_hour,num_depart,diff,station_name,longitude,latitude,total_capacity
0,31000,6.0,2019-05-31,23,0.0,6.0,Eads St & 15th St S,-77.053230,38.858971,14.0
2,31002,4.0,2019-05-31,23,0.0,4.0,Crystal Dr & 20th St S,-77.049232,38.856425,17.0
3,31003,1.0,2019-05-31,23,0.0,1.0,Crystal Dr & 15th St S,-77.049417,38.861056,15.0
4,31004,5.0,2019-05-31,23,0.0,5.0,Aurora Hills Cmty Ctr / 18th St & S Hayes St,-77.059490,38.857866,12.0
5,31005,8.0,2019-05-31,23,0.0,8.0,Pentagon City Metro / 12th St & S Hayes St,-77.059936,38.862303,18.0


### Determine the starting number of bikes for our timeframe

#### Calculate cumulative sum of diff for every station

In [488]:
# Sort the values so that we can view the stations individually one at a time in chronological order
full_bike_data.sort_values(['station_id','Start_day', 'start_hour'], inplace = True)
full_bike_data.reset_index(drop = True, inplace = True)

In [489]:
full_bike_data['cumu_sum1'] = full_bike_data[['station_id','Start_day','start_hour','diff']].groupby(['station_id','Start_day','start_hour']).sum().groupby(level=0).cumsum().reset_index()['diff']

In [490]:
#Some of the numbers are still a little weird - so we will use the custom cumulative sum function
station_num = full_bike_data.loc[0,'station_id']
cumu_sum = 0
zero_counter = 0
cap_counter = 0
for i in range(0,len(full_bike_data)):
    # Check to see if the station number has changed
    if full_bike_data.loc[i,'station_id'] != station_num:
        # If it has, reset the sum to the new station
        station_num = full_bike_data.loc[i,'station_id']
        cumu_sum = 0
    # If the cumu sum has reached max capacity
    if cumu_sum + full_bike_data.loc[i, 'diff'] >= full_bike_data.loc[i,'total_capacity']:
        full_bike_data.at[i,'cumu_sum2'] = full_bike_data.loc[i,'total_capacity']
        cumu_sum = full_bike_data.loc[i,'total_capacity']
        cap_counter +=1
    # If the cumu sum has reached minimum capacity
    elif cumu_sum + full_bike_data.loc[i, 'diff'] < 0:
        full_bike_data.at[i,'cumu_sum2'] = 0
        cumu_sum = 0
        zero_counter += 1
    else:
        # Add to the cumulative sum and add the data point to the table
        cumu_sum += full_bike_data.loc[i, 'diff'] 
        full_bike_data.at[i,'cumu_sum2'] = cumu_sum

### Bike availability error investigation

#### How representative are the two cumulative sum calculations?

In [491]:
print("Number of times restricted cumu sum is negative: {}".format(zero_counter))
print("Number of times restricted cumu sum is greater than capacity: {}".format(cap_counter))
print("Percent of bikes affected by restriction: {}".format((zero_counter + cap_counter)/full_bike_data.shape[0]*100))

Number of times restricted cumu sum is negative: 2360
Number of times restricted cumu sum is greater than capacity: 4678
Percent of bikes affected by restriction: 4.420965350888213


In [492]:
print("Number of times raw cumu sum is greater than capacity: {}".format(full_bike_data[abs(full_bike_data['cumu_sum1']) > full_bike_data['total_capacity']].shape[0]))
print("Number of times raw cumu sum is negative: {}".format(full_bike_data[abs(full_bike_data['cumu_sum1']) < 0].shape[0]))

Number of times raw cumu sum is greater than capacity: 24759
Number of times raw cumu sum is negative: 0


- It looks like we should focus on the restricted cumulative sum since it affects less data points
- The raw cumulative sum indicates that total capacity may not be correct all the time

#### Bad starting points

The main cause behind the errors in the cumulative sum is our inaccurate measure of the starting point
but this only affects **40 stations** in our dataset and the numbers are fairly close so we will assume this is
a natural error that can't be accounted for

In [493]:
num_bad_starting_points = full_bike_data[(full_bike_data['Start_day'] == datetime.date(month = 5, year = 2019, day = 31))&(full_bike_data['num_arrivals']>full_bike_data['total_capacity'])].shape[0]
print("Number of stations with a starting number that is above its capacity: {}".format(num_bad_starting_points))

Number of stations with a starting number that is above its capacity: 40


#### Popular Events

There also appeared to be random **large spikes and dips in arrivals and departures for some stations**, further investigation saw that these events were related to popular events like baseball games <br>
For example, the station near the MLB Nationals Stadium had peak arrivals and departures around the time of the game. <br>
This article shows how this is a common practice for the company, and we can assume that other large spikes are also due to large events <b>
https://www.wnyc.org/story/285378-nine-percent-of-all-d-c-bike-share-bikes-are-at-washington-nationals-stadium-for-the-playoff-game/.

In [494]:
full_bike_data[(full_bike_data['station_id'] == 31648) & (abs(full_bike_data['diff']) > full_bike_data['total_capacity'])]

,station_id,num_arrivals,Start_day,start_hour,num_depart,diff,station_name,longitude,latitude,total_capacity,cumu_sum1,cumu_sum2
133993,31648,39.0,2019-06-01,19,3.0,36.0,Potomac Ave & Half St SW,-77.010815,38.869683,23.0,67.0,23.0
133996,31648,0.0,2019-06-01,22,56.0,-56.0,Potomac Ave & Half St SW,-77.010815,38.869683,23.0,15.0,0.0
134298,31648,35.0,2019-06-26,19,2.0,33.0,Potomac Ave & Half St SW,-77.010815,38.869683,23.0,84.0,23.0
134301,31648,2.0,2019-06-26,22,63.0,-61.0,Potomac Ave & Half St SW,-77.010815,38.869683,23.0,17.0,0.0
134337,31648,34.0,2019-06-29,19,2.0,32.0,Potomac Ave & Half St SW,-77.010815,38.869683,23.0,59.0,23.0
134340,31648,1.0,2019-06-29,22,45.0,-44.0,Potomac Ave & Half St SW,-77.010815,38.869683,23.0,19.0,0.0


### Adding additional information

In [495]:
full_bike_data.drop(columns = ['cumu_sum1'], axis = 1, inplace = True)
full_bike_data.rename({'cumu_sum2':'cumu_sum'}, axis = 1, inplace = True)

In [496]:
full_bike_data['percent_full'] = full_bike_data['cumu_sum']/full_bike_data['total_capacity']
full_bike_data['weekday_num'] = pd.to_datetime(full_bike_data['Start_day'], errors = 'coerce').dt.weekday
#full_bike_data.head()
full_bike_data['weekend_flag'] = np.where((full_bike_data['weekday_num'] == 6) | (full_bike_data['weekday_num'] == 5), 1, 0)

In [497]:
full_bike_data.head()

,station_id,num_arrivals,Start_day,start_hour,num_depart,diff,station_name,longitude,latitude,total_capacity,cumu_sum,percent_full,weekday_num,weekend_flag
0,31000,6.0,2019-05-31,23,0.0,6.0,Eads St & 15th St S,-77.05323,38.858971,14.0,6.0,0.428571,4,0
1,31000,0.0,2019-06-01,8,2.0,-2.0,Eads St & 15th St S,-77.05323,38.858971,14.0,4.0,0.285714,5,1
2,31000,0.0,2019-06-01,10,2.0,-2.0,Eads St & 15th St S,-77.05323,38.858971,14.0,2.0,0.142857,5,1
3,31000,0.0,2019-06-01,14,1.0,-1.0,Eads St & 15th St S,-77.05323,38.858971,14.0,1.0,0.071429,5,1
4,31000,0.0,2019-06-01,17,2.0,-2.0,Eads St & 15th St S,-77.05323,38.858971,14.0,0.0,0.000000,5,1


In [498]:
full_bike_data.to_csv('C:/Users/cbrig/OneDrive/CMU/Perspectives in Data Science/Group Project/cleaned_bike_data_june2019_v2.csv')

## Archive code

In [63]:
# Calculate the last point in time for each station
# idx = may_final_bike_data.groupby(['station_id']).agg({'Start_day': np.max, 'start_hour': np.max, 'index':np.max})['index']
# may_last_bike = may_final_bike_data[may_final_bike_data['index'].isin(idx)]
# may_last_bike = may_last_bike.drop(columns = ['index'])
# print(may_last_bike.shape)
# may_last_bike.head()